In [3]:
"""
---PART ONE---
"""
"""
PSEUDOCODE
create a list of lists of lists which has at the center the puzzle input; 
      around it there're 6 more lines/column for each side. These ones are void('.')
apply conditions
NOTES
structure of the space:
      y[
            x[
                  z[
                        
                  ]
            ]
      ]
"""
import copy

def mk_space(filename):
      #count how long are edges af the cube
      with open(filename, 'r') as fd:
            lenght = len(fd.readline()) - 1

      #create a completely matrix bigger than the one given, with all '.'
      #list contains ys
      #y indexes lines
      #which contain xs
      #x indexes depth
      #which contains zs
      space = []
      for y in range(0, lenght + 12):
            #append a line
            space.append([])
            for x in range(0, lenght + 12):
                  #each element of the line (x) has a depth associated
                  space[y].append([])
                  for z in range(0, 13):
                        #elements in the depth
                        space[y][x].append('.')
      
      with open(filename, 'r') as fd:
            #the first line of the space to fill is line 6
            y = 6
            #central z coordinate
            z = 6
            for line in fd:
                  if line[-1] == '\n':      
                        line = line[0 : -1]
                  i = 0
                  for x in range(6, 6 + lenght):
                        space[y][x][z] = line[i]
                        i += 1
                  y += 1
      return space

def stay_active(space, y, x, z):
      count = 0
      for y_ in range(y - 1, y + 2):
            #to avoid index out of range error
            if y_ < 0 or y_ > (len(space) - 1):
                  continue
            for x_ in range(x - 1, x + 2):
                  if x_ < 0 or x_ > (len(space[y_]) - 1):
                        continue
                  for z_ in range (z - 1, z + 2):
                        if z_ < 0 or z_ > (len(space[y_][x_]) - 1):
                              continue
                        if y_ == y and x_ == x and z_ == z:
                              continue 
                        if space[y_][x_][z_] == '#':
                              count += 1
                        if count > 3:
                              return False
      if count == 2 or count == 3: 
            return True
      else:
            return False

def switch_to_active(space, y, x, z):
      count = 0
      for y_ in range(y - 1, y + 2):
            #to avoid index out of range error
            if y_ < 0 or y_ > (len(space) - 1):
                  continue
            for x_ in range(x - 1, x + 2):
                  if x_ < 0 or x_ > (len(space[y_]) - 1):
                        continue
                  for z_ in range(z - 1, z + 2):
                        if z_ < 0 or z_ > (len(space[y_][x_]) - 1):
                              continue
                        if y_ == y and x_ == x and z_ == z:
                              continue
                        if space[y_][x_][z_] == '#':
                              count += 1
                        if count > 3:
                              return False
      if count == 3:
            return True
      else: 
            return False

def simulation(space):
      #I need prev_space to avoid considering actual cycle
      #changes in changes to make (in other words: all changes are made 
      #looking at the previous cycle, not the one I'm modifying in the 
      #current cycle)
      prev_space = copy.deepcopy(initial_space)
      #for each element check if it switch to active/inactive
      for cycle in range(0, 6):
            prev_space = copy.deepcopy(space)
            '''
            to visualize slices of the space
            print()
            print("CYCLE: ", cycle)        
            for z in range(0, 13):
                  for y in range(0, len(space)):
                        for x in range(0, len(space[y])):
                              print(space[y][x][z], end='')
                        print()
                  print()
            '''

            for y, line in enumerate(initial_space):
                  for x, column in enumerate(initial_space[y]):
                        for z, depth in enumerate(initial_space[y][x]):
                              if space[y][x][z] == '#'  and not stay_active(prev_space, y, x, z):
                                    space[y][x][z] = '.'
                              elif space[y][x][z] == '.' and switch_to_active(prev_space, y, x, z):
                                    space[y][x][z] = '#'
            #print(space)

      count = 0
      for y in range(0, len(space)):
            for x in range(0, len(space[y])):
                  for z in range(0, len(space[y][x])):
                        if space[y][x][z] == '#':
                              count += 1

      return count

filename = '17.txt'
initial_space = mk_space(filename)
print(simulation(copy.deepcopy(initial_space)))

286


In [7]:
def do_part_2():
    coord_dict = {}
    input_filename = "17.txt"
    #input_filename = "sample_input.txt"  # Un-comment this line to use sample data instead
    y = 0
    z = 0
    w = 0
 
    with open(input_filename, 'r') as reader:
        for line in reader:
            this_line = str(line.rstrip())
            for x, c in enumerate(this_line):
                coord_dict[(x, y, z, w)] = c
            y += 1
 
    def neighbor_coord_list(coords: tuple):
        tx = coords[0]
        ty = coords[1]
        tz = coords[2]
        tw = coords[3]
        coord_list = []
        for nx in range(tx - 1, tx + 2):
            for ny in range(ty - 1, ty + 2):
                for nz in range(tz - 1, tz + 2):
                    for nw in range(tw - 1, tw + 2):
                        coord_list.append((nx, ny, nz, nw))
        coord_list.remove((tx, ty, tz, tw))
        return coord_list
 
    def update_map(c_dict: dict):
        neighbor_count = {}
 
        # Find each active coordinate, and update its neighbors
        for this_coord in c_dict:
            if c_dict[this_coord] == '#':
                neighbor_list = neighbor_coord_list(this_coord)
                for this_neighbor in neighbor_list:
                    # If this neighbor hasn't been added, add it
                    try:
                        neighbor_count[this_neighbor] += 1
                    except KeyError:
                        neighbor_count[this_neighbor] = 1
 
        n_map = {}
        active_count = 0
 
        # Go through each coord with active neighbors and see if it needs to change its state
        for this_coord in neighbor_count:
            # If the coord is newly viewed, then its initial state must be inactive.
            try:
                current_state = c_dict[this_coord]
            except KeyError:
                current_state = '.'
 
            # Active coordinates stay active only if they have 2 or 3 active neighbors
            if current_state == '#' and not (2 <= neighbor_count[this_coord] <= 3):
                n_map[this_coord] = '.'
            # Inactive coordinates become active if they have exactly 3 active neighbors
            elif current_state == '.' and neighbor_count[this_coord] == 3:
                n_map[this_coord] = '#'
            # Otherwise, their state remains unchanged
            else:
                n_map[this_coord] = current_state
 
            # Up the count if this coordinate is active
            if n_map[this_coord] == '#':
                active_count += 1
 
        return n_map, active_count
 
    for i in range(6):
        coord_dict, counter = update_map(coord_dict)
    print(counter)

In [8]:
 do_part_2()

960
